# Задание 1. Загрузка данных

Изучить представленный набор данных на основе описания его столбцов, загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.

In [ ]:
import pandas as pd
import numpy as np

# surgery? - column №1 (index 0)
# age - №2 (index 1)
# rectal temperature - №4 (index 3)
# pulse - №5 (4)
# respiratory rate - №6 (5)
# temperature of extremities - №7 (6)
# pain - №11 (10)
# outcome - №23 (22)

def tryInt(strValue):
    try:
        return int(strValue)
    except:
        return np.nan
    
horse_data_base = pd.read_csv(r'https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv', header=None)

horse_data = horse_data_base[[0, 1, 3, 4, 5, 6, 10, 22]]
horse_data.columns = ['surgery', 'age', 'rectal_t', 'pulse', 'respiratory_rate', 't_of_extremities', 'pain', 'outcome']

horse_data['surgery'] = pd.to_numeric(horse_data['surgery'], errors='coerce', downcast='integer')
horse_data['age'] = pd.to_numeric(horse_data['age'], errors='coerce', downcast='integer')
horse_data['rectal_t'] = pd.to_numeric(horse_data['rectal_t'], errors='coerce', downcast='integer')
horse_data['pulse'] = pd.to_numeric(horse_data['pulse'], errors='coerce', downcast='integer')
horse_data['respiratory_rate'] = pd.to_numeric(horse_data['respiratory_rate'], errors='coerce', downcast='integer')
horse_data['t_of_extremities'] = pd.to_numeric(horse_data['t_of_extremities'], errors='coerce', downcast='integer')
horse_data['pain'] = pd.to_numeric(horse_data['pain'], errors='coerce', downcast='integer')
horse_data['outcome'] = pd.to_numeric(horse_data['outcome'], errors='coerce', downcast='integer')

horse_data.info()

# Задание 2. Первичное изучение данных

Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

In [ ]:
def statistics_сontinues_date(sr):
    min_ = sr.min()
    max_ = sr.max()
    range_ = round(max_ - min_, 2)
    disp = round(sr.var(), 2)
    mean = round(sr.mean(), 2)
    median = round(sr.quantile(), 2)
    
    print(f'min={min_}')
    print(f'max={max_}')
    print(f'range={range_}')
    print(f'disp={disp}')
    print(f'mean={mean}')
    print(f'median={median}')
    
    iqr = sr.quantile(0.75) - sr.quantile(0.25)
    lower_bound = round(sr.quantile(0.25) - (1.5 * iqr), 2)
    upper_bound = round(sr.quantile(0.75) + (1.5 * iqr), 2)

    outliers = [x for x in sr if x < lower_bound or x > upper_bound]
    
    print(f'lower_bound={lower_bound}')
    print(f'upper_bound={upper_bound}')
    print(f'iqr={round(iqr, 2)}')
    print(f'outliers={outliers}')
    print()
          
def statistics_nominal_date(sr):
    mode = sr.mode()[0]
    print(f'mode={mode}')
    print()
    
def statistics_ordered_date(sr):
    min_ = sr.min()
    max_ = sr.max()
    range_ = round(max_ - min_, 2)
    disp = round(sr.var(), 2)
    mean = round(sr.mean(), 2)
    median = round(sr.quantile(), 2)
    
    print(f'min={min_}')
    print(f'max={max_}')
    print(f'range={range_}')
    print(f'disp={disp}')
    print(f'mean={mean}')
    print(f'median={median}')
    
    mode = sr.mode()[0]
    print(f'mode={mode}')
    print()
        
# surgery - nominal 
print(f'surgery (nominal)')
statistics_nominal_date(horse_data['surgery'])

# age - nominal
print(f'age (nominal)')
statistics_nominal_date(horse_data['age'])

# rectal_t - continues
print(f'rectal_t (continues)')
statistics_сontinues_date(horse_data['rectal_t'])

# pulse - continues
print(f'pulse (continues)')
statistics_сontinues_date(horse_data['pulse'])

# respiratory_rate - continues
print(f'respiratory_rate (continues)')
statistics_сontinues_date(horse_data['respiratory_rate'])

# t_of_extremities - nominal (hot extremities should correlate with an elevated rectal_t.)
print(f't_of_extremities (nominal)')
statistics_nominal_date(horse_data['t_of_extremities'])

# pain - nominal (ordered?)
print(f'pain (ordered)')
statistics_ordered_date(horse_data['pain'])

# outcome - nominal
print(f'outcome (nominal)')
statistics_nominal_date(horse_data['outcome'])

# Задание 3. Работа с пропусками

Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

In [ ]:
horse_data.info()

In [ ]:
# surgery - nominal. 1 Nan значение - удалим строку.
horse_data_clear = horse_data[horse_data.surgery.notna()]
# age - nominal. 0 NaN значений - ничего не делаем.
# outcome - nominal. 1 NaN значение - удалим строку (но это та же строка, что и с surgery)
horse_data_clear = horse_data_clear[horse_data_clear.outcome.notna()]

# rectal_t - continues. Сувязана с t_of_extremities. NaN в обоих столбцах в 14 строках. Удалим их.
horse_data_clear = horse_data_clear[horse_data_clear.rectal_t.notna() | horse_data_clear.t_of_extremities.notna()]

# Далее посчитаем медину rectal_t для каждого значения t_of_extremities и заполним t_of_extremities медианными значениями
horse_data_clear.rectal_t = horse_data_clear.rectal_t.fillna(horse_data_clear.groupby(['t_of_extremities']).rectal_t.transform('median'))

# Далее посчитаем верхние границы rectal_t для каждого t_of_extremities. Видим, что по границам rectal_t 
# не вычислить t_of_extremities, но одно из значений встречается сильно чаще других. Заполним пропуски модой.
# horse_data_clear.groupby(['t_of_extremities']).agg({'rectal_t' : ['min', 'max', 'mean', 'median']})
# horse_data_clear.groupby(['t_of_extremities']).count()

horse_data_clear.t_of_extremities = horse_data_clear.t_of_extremities.fillna(horse_data_clear.t_of_extremities.mode()[0])

# respiratory_rate - continues. Имеет большой разброс значений и не показательный параметр. А так же имеет много пропусков. 
# Удалим столбец
horse_data_clear = horse_data_clear.drop(columns='respiratory_rate') 

# pulse - nominal. Видим, что чем больше боль, чем выше пульс. Заполним пропуски медианой по групе боли.
horse_data_clear.pulse = horse_data_clear.pulse.fillna(horse_data_clear.groupby(['pain']).pulse.transform('median'))

# pain - не понятно, как вычислить. Есть связь с surgery и пульсом, но весьма не одназначная. Строк c NaN осталось 44. 
# Удалим эти строки

horse_data_clear = horse_data_clear[horse_data_clear.pain.notna()]
horse_data_clear.info()